In [1]:
from thesis_library import *

In [5]:
def build_ss_loss_function(date):
    
    market_prices = bonds_prices[date:date].T
    market_prices.columns = ['Market prices']
    print(market_prices)
    
    payments_on_date = bonds_payments[bonds_payments['Дата фактической выплаты'] >= date]
    print(payments_on_date)
    
    def ss_loss_function(theta):
        
        C = np.ones(8) * 1000
        
        nonlocal payments_on_date, market_prices, date
    
        calc_df = pd.concat([(payments_on_date['Дата фактической выплаты'] - date).apply(lambda x: x.days)/365, 
                          payments_on_date[['Торговый код', 'Сумма купона, RUB', 'Погашение номинала, RUB']]], axis = 1)
        
        discount(calc_df, theta)
      
        calc_prices = pd.DataFrame(calc_df.groupby('Торговый код')['Discounted'].sum())
        
        result_df = pd.concat([pd.DataFrame(calc_df.groupby('Торговый код')['Discounted'].sum()), market_prices], axis = 1)
        
        #Sum of squares
        J = (((np.array(result_df['Discounted']) - np.array(result_df['Market prices']))/1000)**2).sum()
        
        #constraints
        if (theta[0] > 0) & (theta[0] <= 30):
            c1 = 0
        else:
            c1 = C[1] * theta[0]**2

        if (theta[1] > 0) & (theta[1] <= 30):
            c2 = 0
        else:
            c2 = C[2] * theta[1]**2
            
        if theta[2] + theta[3] >= 0:
            c3 = 0
        else:
            c3 = C[3] * (theta[2] + theta[3])**2
            
        if (theta[2] >= 0) & (theta[2] <= 100):
            c4 = 0
        else:
            c4 = C[4] * theta[2]**2
        
        if (theta[3] >= -100) & (theta[3] <= 100):
            c5 = 0
        else:
            c5 = C[5] * theta[3]**2 

        if (theta[4] >= -100) & (theta[4] <= 100):
            c6 = 0
        else:
            c6 = C[6] * theta[4]**2 
            
        if (theta[5] >= -100) & (theta[5] <= 100):
            c7 = 0
        else:
            c7 = C[7] * theta[5]**2 
                            
        return J + c1 + c2 + c3 + c4 + c5 + c6 + c7
    
    return ss_loss_function

In [2]:
date = dates[0]

In [3]:
market_prices = bonds_prices[date:date].T
market_prices.columns = ['Market prices']
market_prices.dropna(inplace=True)

payments_on_date = bonds_payments[bonds_payments['Дата фактической выплаты'] >= date]

In [4]:
market_prices.index.tolist()

['SU26205RMFS3',
 'SU26207RMFS9',
 'SU26208RMFS7',
 'SU26212RMFS9',
 'SU26215RMFS2']

In [5]:
bonds_payments

,Бумага,ISIN,Торговый код,Номер платежа,Дата фактической выплаты,"Ставка купона, % годовых","Сумма купона, RUB","Погашение номинала, RUB"
0,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,1,2011-10-26,"7,60","40,81",NaN
1,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,2,2012-04-25,"7,60","37,90",NaN
2,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,3,2012-10-24,"7,60","37,90",NaN
3,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,4,2013-04-24,"7,60","37,90",NaN
4,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,5,2013-10-23,"7,60","37,90",NaN
5,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,6,2014-04-23,"7,60","37,90",NaN
6,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,7,2014-10-22,"7,60","37,90",NaN
7,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,8,2015-04-22,"7,60","37,90",NaN
8,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,9,2015-10-21,"7,60","37,90",NaN
9,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,10,2016-04-20,"7,60","37,90",NaN


In [9]:
payments_on_date[payments_on_date['Торговый код'].isin(market_prices.index)]

,Бумага,ISIN,Торговый код,Номер платежа,Дата фактической выплаты,"Ставка купона, % годовых","Сумма купона, RUB","Погашение номинала, RUB"
9,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,10,2016-04-20,"7,60","37,90",NaN
10,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,11,2016-10-19,"7,60","37,90",NaN
11,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,12,2017-04-19,"7,60","37,90",NaN
12,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,13,2017-10-18,"7,60","37,90",NaN
13,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,14,2018-04-18,"7,60","37,90",NaN
14,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,15,2018-10-17,"7,60","37,90",NaN
15,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,16,2019-04-17,"7,60","37,90",NaN
16,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,17,2019-10-16,"7,60","37,90",NaN
17,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,18,2020-04-15,"7,60","37,90",NaN
18,"Россия, 26205",RU000A0JREQ7,SU26205RMFS3,19,2020-10-14,"7,60","37,90",NaN


# Previous day values

# Random starting values

In [6]:
theta1 = optimize_on_day_with_starting_values(date_number=0, method='BFGS', theta0=np.array([2, 5, 2, 6, 1, 4]))

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

# Diebold, Li starting values

Надо сгенерировать список функций, сохранить, а потом с ним работать.